### Basic Statistics -2

In [2]:
import pandas as pd
import numpy as np

In [3]:
patient_df = pd.read_csv("Patient_data.csv")

In [4]:
patient_df.info()
patient_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45


In [5]:
billing_cols = ['PatientID', 'Department', 'Doctor', 'BillAmount']
patient_billing_df = patient_df[billing_cols]
patient_billing_df.head()

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


In [6]:
cleaned_patient_df = patient_df.drop(columns=['ReceptionistID', 'CheckInTime'])
cleaned_patient_df.head()

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


In [7]:
dept_bill_summary = patient_billing_df.groupby('Department')['BillAmount'].sum().reset_index()
dept_bill_summary

,Department,BillAmount
0,Cardiology,16200.0
1,Dermatology,0.0
2,Neurology,0.0
3,Orthopedics,7500.0


In [8]:
unique_patient_df = patient_billing_df.drop_duplicates(subset='PatientID')
unique_patient_df.head()

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


In [9]:
mean_bill = unique_patient_df['BillAmount'].mean()
unique_patient_df['BillAmount'] = unique_patient_df['BillAmount'].fillna(mean_bill)
unique_patient_df.head()

C:\Users\shrut\AppData\Local\Temp\ipykernel_17024\1366904101.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_patient_df['BillAmount'] = unique_patient_df['BillAmount'].fillna(mean_bill)


,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.000000
1,102,Neurology,Dr. John,6233.333333
2,103,Orthopedics,Dr. Lee,7500.000000
3,104,Cardiology,Dr. Smith,6200.000000
4,105,Dermatology,Dr. Rose,6233.333333


In [10]:
billing_df = pd.read_csv("Billing_data.csv")
billing_df.head

<bound method NDFrame.head of    PatientID  InsuranceCovered  FinalAmount
0        101              2000         3000
1        102              1500         3500
2        103              2500         5000
3        104              3000         3200
4        105              1000         4000>

In [11]:
merged_df = pd.merge(unique_patient_df, billing_df, on='PatientID')
merged_df.head()

,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Cardiology,Dr. Smith,5000.000000,2000,3000
1,102,Neurology,Dr. John,6233.333333,1500,3500
2,103,Orthopedics,Dr. Lee,7500.000000,2500,5000
3,104,Cardiology,Dr. Smith,6200.000000,3000,3200
4,105,Dermatology,Dr. Rose,6233.333333,1000,4000


In [12]:
new_patients = pd.DataFrame({
    'PatientID': [205, 207],
    'Department': ['Cardiology', 'Neurology'],
    'Doctor': ['Dr. Shruti', 'Dr. sagar'],
    'BillAmount': [50000, 7000]
})

In [13]:
updated_df = pd.concat([merged_df, new_patients], axis=0, ignore_index=True)
updated_df.tail()

,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
2,103,Orthopedics,Dr. Lee,7500.000000,2500.0,5000.0
3,104,Cardiology,Dr. Smith,6200.000000,3000.0,3200.0
4,105,Dermatology,Dr. Rose,6233.333333,1000.0,4000.0
5,205,Cardiology,Dr. Shruti,50000.000000,NaN,NaN
6,207,Neurology,Dr. sagar,7000.000000,NaN,NaN


In [16]:
updated_df = updated_df.drop(columns=['InsuranceCovered', 'FinalAmount'], errors='ignore')

In [17]:
new_cols = pd.DataFrame({
    'InsuranceCovered': ([True, False] * (len(updated_df)//2 + 1))[:len(updated_df)],
    'FinalAmount': updated_df['BillAmount'] * 0.9
})

In [18]:
final_df = pd.concat([updated_df.reset_index(drop=True), new_cols.reset_index(drop=True)], axis=1)
final_df.head()

,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Cardiology,Dr. Smith,5000.000000,True,4500.0
1,102,Neurology,Dr. John,6233.333333,False,5610.0
2,103,Orthopedics,Dr. Lee,7500.000000,True,6750.0
3,104,Cardiology,Dr. Smith,6200.000000,False,5580.0
4,105,Dermatology,Dr. Rose,6233.333333,True,5610.0
